In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
item_prices = pd.read_csv('../data/raw/item_prices.csv')
item_sales = pd.read_csv('../data/raw/item_sales.csv')
calendar = pd.read_csv('../data/raw/daily_calendar_with_events.csv')

In [3]:
regions = item_sales.groupby('region')
all_regions_distinct = list(item_sales['region'].unique())
itens = item_sales.groupby('item')
all_itens_distinct = list(item_sales['item'].unique())
total_days = len([i for i in item_sales if i.startswith('d_')])

In [6]:
# item_sales.head(10)
all_regions_distinct

['New York', 'Boston', 'Philadelphia']

In [31]:
total_days

1913

In [11]:
melted = item_sales.melt(id_vars=['item', 'category', 'department', 'store_code', 'region'], value_vars=[i for i in item_sales if i.startswith('d_')], var_name=['days'], value_name='sales')
merged = pd.merge(melted, calendar, how='inner', left_on = 'days', right_on = 'd')
merged.head(10)

,item,category,department,store_code,region,days,sales,date,weekday,weekday_int,d,event
0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
2,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
3,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
4,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
5,ACCESORIES_1_006,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
6,ACCESORIES_1_007,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN
7,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,12,2011-01-29,Saturday,1,d_1,NaN
8,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,2,2011-01-29,Saturday,1,d_1,NaN
9,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,d_1,NaN


In [12]:
merged['date'] = pd.to_datetime(merged['date'])
merged['year'] = merged['date'].dt.year
merged['month'] = merged['date'].dt.month
merged['day'] = merged['date'].dt.day

In [13]:
merged['week'] = merged['date'].dt.isocalendar().week.apply(str)
merged['week'] = merged['week'].apply(lambda x: x.zfill(2))
merged['yearweek'] = merged['year'].apply(str) + merged['week']

In [14]:
merged['item'] = merged['item'].astype('string')
merged['category'] = merged['category'].astype('string')
merged['store_code'] = merged['store_code'].astype('string')
merged['yearweek'] = merged['yearweek'].astype('string')

In [20]:
item_prices['yearweek'] = item_prices['yearweek'].astype('Int64')
for i in range(len(item_prices)):
   if pd.isnull(item_prices['yearweek'][i]):
       item_prices['yearweek'][i]=item_prices['yearweek'][i-1]+1

In [21]:
item_prices['item'] = item_prices['item'].astype('string')
item_prices['category'] = item_prices['category'].astype('string')
item_prices['store_code'] = item_prices['store_code'].astype('string')
item_prices['yearweek'] = item_prices['yearweek'].astype('Int64').astype('string')
merged2 = merged.merge(item_prices, on=['item', 'category', 'store_code', 'yearweek'], how='left')

In [27]:
merged3 = merged2[merged2['sell_price'].notna()]

In [30]:
for region in all_regions_distinct:
    print(region)
    print(len(merged3[merged3['region'] == region]))

New York
18126090
Boston
14015811
Philadelphia
13652740


https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65/notebook

In [3]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [4]:
merged3, NAlist = reduce_mem_usage(merged3)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 6288.937026977539  MB
******************************
Column:  sales
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  weekday_int
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  year
dtype before:  int64
dtype after:  uint16
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  day
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  week
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  yearweek
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  sell_price
dtype before:  float64
dtype after:  float32
******

https://towardsdatascience.com/still-saving-your-data-in-csv-try-these-other-options-9abe8b83db3a

In [6]:
merged3.to_pickle('../data/processed/full_data.pkl')